In [ ]:
import requests
from bs4 import BeautifulSoup 
import arxiv
import urllib
import time
from tqdm.notebook import tqdm
import pickle
import sys
import os
sys.path.append('../')
from post_discord import post_discord

In [3]:
# 1000個の論文を全部DL 実行注意
# arxivの規約で3秒に1つまでならDLするのは認められているっぽい（5秒待つ）

post_discord("start arxiv download... (arxiv_pdf_downloader_magnetar.ipynb)")

# 中に2個撤回されたものがあった
search = arxiv.Search(
  query = '"magnetar" OR "fast radio burst"',
  max_results = 1002,
)
for t, result in enumerate(tqdm(list(search.results()))):
    for i in range(10):
        try:
            result.download_pdf(dirpath="./arxiv_papers_magnetar_OR_fast_radio_burst")
            time.sleep(5) # 負荷対策
            break
        # たまに TLS/SSL connection has been closed が起こるので再試行
        except Exception as e:
            if i==9:
                raise e
            post_discord("{}: {}".fotmat(t,str(e)))
            time.sleep(10)            

post_discord("finish arxiv download (arxiv_pdf_downloader_magnetar.ipynb)")

  0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import tiktoken
import os

In [16]:
# 参考 https://stackoverflow.com/questions/76886954/multiple-file-loading-and-embeddings-with-openai
pdf_folder_path = "./arxiv_papers_magnetar_OR_fast_radio_burst"
#print(os.listdir(pdf_folder_path))
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

all_documents = []

post_discord("start splitter (arxiv_pdf_downloader_magnetar.ipynb)")

with tqdm(loaders) as pbar:
    for loader in pbar:
        try:
            pbar.set_description("[Loading raw document... {}]".format(loader.file_path))
            raw_documents = loader.load()
            splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator=". ") # Splitting text
            documents = splitter.split_documents(raw_documents)
            all_documents.extend(documents)
        except Exception as e:
            post_discord("error occured at {}, skipped (arxiv_pdf_downloader_magnetar.ipynb)\n {}".format(loader.file_path,str(e)))

post_discord("finish splitter (arxiv_pdf_downloader_magnetar.ipynb)")

  0%|          | 0/1001 [00:00<?, ?it/s]

Created a chunk of size 1412, which is longer than the specified 1000
Created a chunk of size 1100, which is longer than the specified 1000
Created a chunk of size 1053, which is longer than the specified 1000
Created a chunk of size 1111, which is longer than the specified 1000
Created a chunk of size 1137, which is longer than the specified 1000
Created a chunk of size 1046, which is longer than the specified 1000
Created a chunk of size 2743, which is longer than the specified 1000
Created a chunk of size 1429, which is longer than the specified 1000
Created a chunk of size 1062, which is longer than the specified 1000
Created a chunk of size 2268, which is longer than the specified 1000
Created a chunk of size 1493, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 1668, which is longer than the specified 1000
Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of s

In [17]:
encoding = tiktoken.encoding_for_model("text-embedding-ada-002")
text = ""
for doc in all_documents:
    text += doc.page_content.replace("\n", " ")
token_count = len(encoding.encode(text, allowed_special='all'))
print("予想価格: {} USD".format(token_count*0.0001/1000)) # https://openai.com/pricing > Embedding models > Model: Ada v2; Usage: $0.0001 / 1K tokens

post_discord("予想価格: {} USD".format(token_count*0.0001/1000))

予想価格: 1.6448356000000002 USD


In [18]:
#with open('all_documents_magnetar_OR_fast_radio_burst.pkl', 'wb') as f:
#    pickle.dump(all_documents, f)

#with open('all_documents_magnetar_OR_fast_radio_burst.pkl', 'rb') as f:
#    all_documents = pickle.load(f)

In [42]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
# OpenAI APIを使う処理 お金がかかる
# 一発でやろうとするとapi規制に引っかかるので何回かに分ける必要がある

limit = 1000000
end = len(all_documents)

step = int(limit*0.7//(token_count//len(all_documents)))

post_discord("start embedding (arxiv_pdf_downloader_magnetar.ipynb)")

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(all_documents[0:step], embeddings)
time.sleep(60)

for i in tqdm(range(1,end//step+1)):
    db2 = FAISS.from_documents(all_documents[step*i:min(step*(i+1),end)], embeddings)
    db.merge_from(db2)
    db.save_local("faiss_index_1000_magnetar_OR_fast_radio_burst")
    time.sleep(60)

post_discord("finish embedding (arxiv_pdf_downloader_magnetar.ipynb)")

  0%|          | 0/23 [00:00<?, ?it/s]

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: HTTP code 520 from API (<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="